<a href="https://colab.research.google.com/github/arutraj/.githubcl/blob/main/Spark_SQL_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install pyspark and customize Colab configuration


The pyspark installation will persist until the runtime is recycled.

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,071 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Ign:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

'/usr/local/lib/python3.10/dist-packages/pyspark'

# Initialize Spark

In [2]:
import pyspark
from pyspark import SparkContext

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

sc = spark.sparkContext
spark

# Load Cars Dataset

In [3]:
!wget https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv

--2024-10-27 18:33:00--  https://jacobceles.github.io/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobceles.github.io (jacobceles.github.io)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobceles.github.io (jacobceles.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv [following]
--2024-10-27 18:33:00--  https://jacobcelestine.com/knowledge_repo/colab_and_pyspark/cars.csv
Resolving jacobcelestine.com (jacobcelestine.com)... 185.199.108.153, 185.199.109.153, 185.199.110.153, ...
Connecting to jacobcelestine.com (jacobcelestine.com)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22608 (22K) [text/csv]
Saving to: ‘cars.csv’

cars.csv            100%[===================>]  22.08K  --.-KB/s    in 0s      

2024-10-27 18:33:00 (157 MB/s) - ‘cars.csv’ saved [22608/22608]



In [4]:
# Using Apache Spark to read the CSV file into a DataFrame
df = df = spark.read.csv('cars.csv', header=True, sep=";")
df.createOrReplaceTempView("cars")

In [5]:
df.show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0| 4354.|         9.0|   70|    US|
|   Plymouth Fury iii|14.0|        8|       440.0|


# Common Table Expression (CTE)


In [6]:
# Using CTE to filter out those cars with MPG>=15

spark.sql("""
  WITH subset_cars AS (
    SELECT * FROM cars WHERE MPG>=15.0
  )

  SELECT * FROM subset_cars
""").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|   Buick Skylark 320|15.0|        8|       350.0|     165.0| 3693.|        11.5|   70|    US|
|  Plymouth Satellite|18.0|        8|       318.0|     150.0| 3436.|        11.0|   70|    US|
|       AMC Rebel SST|16.0|        8|       304.0|     150.0| 3433.|        12.0|   70|    US|
|         Ford Torino|17.0|        8|       302.0|     140.0| 3449.|        10.5|   70|    US|
|    Ford Galaxie 500|15.0|        8|       429.0|     198.0| 4341.|        10.0|   70|    US|
|  AMC Ambassador DPL|15.0|        8|       390.0|     190.0| 3850.|         8.5|   70|    US|
| Dodge Challenger SE|15.0|        8|       383.0|

In [7]:
# subset_mpg: This CTE selects all rows from the cars table where the MPG (miles per gallon) is greater than or equal to 15.0.
# subset_hp: This CTE selects all rows from the cars table where the Horsepower is greater than or equal to 100.0.
# After creating these subsets, the main query then performs a UNION operation to combine the results of two separate queries:

spark.sql("""
  WITH subset_mpg AS (
    SELECT * FROM cars WHERE MPG>=15.0
  ),

  subset_hp AS (
    SELECT * FROM cars WHERE Horsepower>=100.0
  )

  SELECT "MPG>=15.0" AS condition, COUNT(*) AS num_cars  FROM subset_mpg
  UNION
  SELECT "Horsepower>=100.0" AS condition, COUNT(*) AS num_cars FROM subset_hp
""").show()

+-----------------+--------+
|        condition|num_cars|
+-----------------+--------+
|        MPG>=15.0|     345|
|Horsepower>=100.0|     174|
+-----------------+--------+



# Subquery in WHERE clause

In [8]:
# This query retrieves rows from the cars table where the MPG value is above the average MPG value of all cars in the table.
spark.sql("""
  SELECT * FROM cars WHERE MPG > (SELECT AVG(MPG) FROM cars)
""").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Toyota Corolla Ma...|24.0|        4|       113.0|     95.00| 2372.|        15.0|   70| Japan|
|        Datsun PL510|27.0|        4|       97.00|     88.00| 2130.|        14.5|   70| Japan|
|Volkswagen 1131 D...|26.0|        4|       97.00|     46.00| 1835.|        20.5|   70|Europe|
|         Peugeot 504|25.0|        4|       110.0|     87.00| 2672.|        17.5|   70|Europe|
|         Audi 100 LS|24.0|        4|       107.0|     90.00| 2430.|        14.5|   70|Europe|
|            Saab 99e|25.0|        4|       104.0|     95.00| 2375.|        17.5|   70|Europe|
|            BMW 2002|26.0|        4|       121.0|     113.0| 2234.|        12.5|   70|Europe|
|        Datsun PL510|27.0|        4|       97.00|

# Aggregate Queries

In [9]:
# Using GROUP BY and MAX, AVG aggregate functions to calculate the average MPG and the maximum Horsepower for each region
spark.sql("""
  SELECT Origin, ROUND(AVG(MPG), 2) AS average_mpg, MAX(Horsepower) AS max_horsepower FROM cars GROUP BY Origin
""").show()

+------+-----------+--------------+
|Origin|average_mpg|max_horsepower|
+------+-----------+--------------+
|Europe|      26.75|         98.00|
| Japan|      30.45|         97.00|
|    US|      19.69|         98.00|
+------+-----------+--------------+



In [10]:
# Using HAVING clause in the previous query
spark.sql("""
  SELECT model, ROUND(AVG(MPG), 2) AS average_mpg, MAX(Horsepower) AS max_horsepower FROM cars GROUP BY model HAVING average_mpg>=20.0
""").show()

+-----+-----------+--------------+
|model|average_mpg|max_horsepower|
+-----+-----------+--------------+
|   71|      20.52|         95.00|
|   74|       22.7|         97.00|
|   75|      20.27|         98.00|
|   76|      21.57|         95.00|
|   77|      23.38|         98.00|
|   78|      24.06|         97.00|
|   79|      25.09|         90.00|
|   80|       33.7|         92.00|
|   81|      29.32|         92.00|
|   82|      31.71|         96.00|
+-----+-----------+--------------+



In [11]:
# Using ORDER BY clause in the previous query
spark.sql("""
  SELECT Origin, ROUND(AVG(MPG), 2) AS average_mpg, MAX(Horsepower) AS max_horsepower FROM cars GROUP BY Origin ORDER BY average_mpg DESC
""").show()

+------+-----------+--------------+
|Origin|average_mpg|max_horsepower|
+------+-----------+--------------+
| Japan|      30.45|         97.00|
|Europe|      26.75|         98.00|
|    US|      19.69|         98.00|
+------+-----------+--------------+



# JOIN

In [12]:
# Using INNER JOIN to filter out cars with MPG>=15.0 and Horsepower>=100.0.
# For this we created 2 CTEs and then joined them

spark.sql("""
  WITH subset_mpg AS (
    SELECT * FROM cars WHERE MPG>=15.0
  ),

  subset_hp AS (
    SELECT * FROM cars WHERE Horsepower>=100.0
  )

  SELECT DISTINCT subset_mpg.* FROM subset_mpg JOIN subset_hp ON subset_mpg.car = subset_hp.car
""").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|
+--------------------+----+---------+------------+----------+------+------------+-----+------+
|Chevrolet Caprice...|17.0|        8|       305.0|     130.0| 3840.|        15.4|   79|    US|
|       Buick Skylark|26.6|        4|       151.0|     84.00| 2635.|        16.4|   81|    US|
|Chevrolet Monte C...|15.0|        8|       350.0|     145.0| 4082.|        13.0|   73|    US|
|         AMC Matador|16.0|        6|       258.0|     110.0| 3632.|        18.0|   74|    US|
|Mercury Cougar Br...|15.0|        8|       302.0|     130.0| 4295.|        14.9|   77|    US|
|           Saab 99LE|25.0|        4|       121.0|     115.0| 2671.|        13.5|   75|Europe|
|Chevrolet Chevell...|18.0|        8|       307.0|     130.0| 3504.|        12.0|   70|    US|
|       Buick Century|22.4|        6|       231.0|

# WINDOW Functions

In [13]:
# Using window function with AVG aggregation to find out average horsepower for each region

spark.sql("SELECT *, ROUND(AVG(Horsepower) OVER (PARTITION BY Origin), 2) AS avg_horsepower FROM cars ORDER BY RAND()").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+--------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|avg_horsepower|
+--------------------+----+---------+------------+----------+------+------------+-----+------+--------------+
|         Dodge Aspen|18.6|        6|       225.0|     110.0| 3620.|        18.7|   78|    US|        118.01|
|  Mercedes-Benz 240d|30.0|        4|       146.0|     67.00| 3250.|        21.8|   80|Europe|         78.78|
|         Peugeot 504|27.2|        4|       141.0|     71.00| 3190.|        24.8|   79|Europe|         78.78|
|    Ford Galaxie 500|14.0|        8|       351.0|     153.0| 4129.|        13.0|   72|    US|        118.01|
|Peugeot 505s Turb...|28.1|        4|       141.0|     80.00| 3230.|        20.4|   81|Europe|         78.78|
|Toyota Corolla Te...|38.1|        4|       89.00|     60.00| 1968.|        18.8|   80| Japan|         79.84|
|    Plymo

In [14]:
# Using DENSE_RANK() to rank each car in decreasing order of Horsepower & MPG

spark.sql("SELECT *, DENSE_RANK() OVER (ORDER BY Horsepower DESC, MPG DESC) AS rank FROM cars WHERE Horsepower>0.0").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+----+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|rank|
+--------------------+----+---------+------------+----------+------+------------+-----+------+----+
|         Volvo 244DL|22.0|        4|       121.0|     98.00| 2945.|        14.5|   75|Europe|   1|
|        Ford Grenada|18.5|        6|       250.0|     98.00| 3525.|        19.0|   77|    US|   2|
|          Datsun 510|27.2|        4|       119.0|     97.00| 2300.|        14.7|   78| Japan|   3|
|         Honda Civic|24.0|        4|       120.0|     97.00| 2489.|        15.0|   74| Japan|   4|
|          Datsun 710|24.0|        4|       119.0|     97.00| 2545.|        17.0|   75| Japan|   4|
|       Datsun 200-SX|23.9|        4|       119.0|     97.00| 2405.|        14.9|   78| Japan|   5|
|Toyota Corolla Ma...|23.0|        4|       120.0|     97.00| 2506.|        14.5|   72| Japan|   6|


# User Defined Functions

In [16]:
# Define python function to calculate power to weight ratio
def get_power_weight_ratio(hp, weight):
  return float(hp) / float(weight)

# Register UDF to be used with SQL
spark.udf.register("get_power_weight_ratio", get_power_weight_ratio, T.DoubleType())

# Using UDF to create a new column and sorting in descending orders
spark.sql("SELECT *, ROUND(get_power_weight_ratio(Horsepower, Weight), 3) AS power_weight_ratio FROM cars ORDER BY power_weight_ratio DESC").show()

+--------------------+----+---------+------------+----------+------+------------+-----+------+------------------+
|                 Car| MPG|Cylinders|Displacement|Horsepower|Weight|Acceleration|Model|Origin|power_weight_ratio|
+--------------------+----+---------+------------+----------+------+------------+-----+------+------------------+
|Buick Estate Wago...|14.0|        8|       455.0|     225.0| 3086.|        10.0|   70|    US|             0.073|
|  Pontiac Grand Prix|16.0|        8|       400.0|     230.0| 4278.|        9.50|   73|    US|             0.054|
|    Chevrolet Impala|14.0|        8|       454.0|     220.0| 4354.|         9.0|   70|    US|             0.051|
|    Pontiac Catalina|14.0|        8|       455.0|     225.0| 4425.|        10.0|   70|    US|             0.051|
|            BMW 2002|26.0|        4|       121.0|     113.0| 2234.|        12.5|   70|Europe|             0.051|
|   Plymouth Fury iii|14.0|        8|       440.0|     215.0| 4312.|         8.5|   70| 



---

